In [19]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [41]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [25]:
pd.options.display.max_columns=100
df.loc[54012:54013]

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
54012,2021-22,1610612744,GSW,Golden State Warriors,0022100169,2021-11-10,GSW vs. MIN,W,240,40,85,0.471,11,40,0.275,32,40,0.80,20,33,53,23,14,6,16,21,123,13,124.7,121.8,106.0,110.0,18.8,11.8,0.575,1.44,16.2,0.489,0.600,0.551,16.227,15.8,0.535,0.599,1.0,0.204,101.2,100.5,83.75,101.0,0.607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54013,2021-22,1610612750,MIN,Minnesota Timberwolves,0022100169,2021-11-10,MIN @ GSW,L,240,39,96,0.406,17,44,0.386,15,20,0.75,18,23,41,30,8,4,17,28,110,-13,106.0,110.0,124.7,121.8,-18.8,-11.8,0.769,1.76,19.8,0.400,0.511,0.449,16.378,17.0,0.495,0.525,1.0,0.200,101.2,100.5,83.75,100.0,0.393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
home_team = 'PHI'
away_team = 'TOR'
home_spread = -3
away_spread = -home_spread
home_ml = 1.69
away_ml = 2.25

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

matchup

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 32.54it/s]
progress:: 334it [00:00, 4839.74it/s]


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,AST_team_diff,STL_team_diff,BLK_team_diff,TOV_team_diff,PF_team_diff,PTS_team_diff,PLUS_MINUS_team_diff,E_OFF_RATING_team_diff,E_DEF_RATING_team_diff,E_NET_RATING_team_diff,POSS_team_diff,PIE_team_diff,PTS_2PT_MR_team_diff,PTS_FB_team_diff,PTS_OFF_TOV_team_diff,PTS_PAINT_team_diff,AST_2PM_team_diff,AST_3PM_team_diff,UAST_2PM_team_diff,UAST_3PM_team_diff,ATS_DIFF_team_diff,RECORD_opp_diff,FG2M_opp_diff,FG2A_opp_diff,FG3M_opp_diff,FG3A_opp_diff,FTM_opp_diff,FTA_opp_diff,OREB_opp_diff,DREB_opp_diff,REB_opp_diff,AST_opp_diff,STL_opp_diff,BLK_opp_diff,TOV_opp_diff,PF_opp_diff,PTS_opp_diff,PLUS_MINUS_opp_diff,E_OFF_RATING_opp_diff,E_DEF_RATING_opp_diff,E_NET_RATING_opp_diff,POSS_opp_diff,PIE_opp_diff,PTS_2PT_MR_opp_diff,PTS_FB_opp_diff,PTS_OFF_TOV_opp_diff,PTS_PAINT_opp_diff,AST_2PM_opp_diff,AST_3PM_opp_diff,UAST_2PM_opp_diff,UAST_3PM_opp_diff,ATS_DIFF_opp_diff,TEAM_COVERED_opp_diff,WL_PCT_diff,ATS_PCT_diff,AVG_SPREAD_diff,AVG_ML_diff,AVG_ATS_DIFF_diff,REST_diff,OREB_PCT_team_diff,OREB_PCT_opp_diff,DREB_PCT_team_diff,DREB_PCT_opp_diff,REB_PCT_team_diff,REB_PCT_opp_diff,TS_PCT_team_diff,TS_PCT_opp_diff,EFG_PCT_team_diff,EFG_PCT_opp_diff,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,0.312456,-1.613421,-10.672206,2.20517,1.016316,2.374622,1.917411,-5.133234,1.33014,-3.803094,3.557965,-2.526088,1.166804,1.797896,-0.342819,5.763291,6.775018,8.048504,-1.100178,9.123452,-0.17984,0.053919,-1.150881,-0.227798,-5.081697,-1.922911,2.36765,1.142074,-4.070228,1.200738,3.401381,-0.312456,2.874346,9.642801,-2.289549,-3.19131,0.108229,0.261229,3.045944,-1.328255,1.717689,-0.471471,1.861381,-2.1731,-3.71255,0.993709,-1.011727,-6.775018,-1.100178,8.048504,-9.123452,0.136468,-0.053919,1.42271,4.797576,1.048574,4.147682,1.738169,-2.153673,1.485062,-0.407647,-3.401381,-0.238146,0.254545,0.154545,-3.9,-0.36561,1.690909,1.0,-0.083087,0.04419,-0.04419,0.083087,-0.03206,0.03206,0.09151,-0.047903,0.078896,-0.045702,3.746594,-0.524292,2.435227,-3.781198,0.795054,1657.702834,1491.199582,8.41,-3.0,1.69
1,-0.312456,1.613421,10.672206,-2.20517,-1.016316,-2.374622,-1.917411,5.133234,-1.33014,3.803094,-3.557965,2.526088,-1.166804,-1.797896,0.342819,-5.763291,-6.775018,-8.048504,1.100178,-9.123452,0.17984,-0.053919,1.150881,0.227798,5.081697,1.922911,-2.36765,-1.142074,4.070228,-1.200738,-3.401381,0.312456,-2.874346,-9.642801,2.289549,3.19131,-0.108229,-0.261229,-3.045944,1.328255,-1.717689,0.471471,-1.861381,2.1731,3.71255,-0.993709,1.011727,6.775018,1.100178,-8.048504,9.123452,-0.136468,0.053919,-1.42271,-4.797576,-1.048574,-4.147682,-1.738169,2.153673,-1.485062,0.407647,3.401381,0.238146,-0.254545,-0.154545,3.9,0.36561,-1.690909,-1.0,0.083087,-0.04419,0.04419,-0.083087,0.03206,-0.03206,-0.09151,0.047903,-0.078896,0.045702,-3.746594,0.524292,-2.435227,3.781198,0.204946,1491.199582,1657.702834,-8.41,3.0,2.25


In [39]:
full_df.loc[full_df['TEAM_ABBREVIATION_team'] == 'TOR']

,SEASON_team,TEAM_ID_team,TEAM_ABBREVIATION_team,TEAM_NAME_team,GAME_ID,GAME_DATE_team,MATCHUP_team,HOME_GAME_team,TEAM_SCORE_team,POINT_DIFF_team,WL_team,ML_team,SPREAD_team,TEAM_COVERED_team,RECORD_team,FG2M_team,FG2A_team,FG3M_team,FG3A_team,FTM_team,FTA_team,OREB_team,DREB_team,REB_team,AST_team,STL_team,BLK_team,TOV_team,PF_team,PTS_team,PLUS_MINUS_team,E_OFF_RATING_team,E_DEF_RATING_team,E_NET_RATING_team,POSS_team,PIE_team,PTS_2PT_MR_team,PTS_FB_team,PTS_OFF_TOV_team,PTS_PAINT_team,AST_2PM_team,AST_3PM_team,UAST_2PM_team,UAST_3PM_team,ATS_DIFF_team,RECORD_opp,FG2M_opp,FG2A_opp,FG3M_opp,FG3A_opp,...,DREB_opp,REB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp,PLUS_MINUS_opp,E_OFF_RATING_opp,E_DEF_RATING_opp,E_NET_RATING_opp,POSS_opp,PIE_opp,PTS_2PT_MR_opp,PTS_FB_opp,PTS_OFF_TOV_opp,PTS_PAINT_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp,ATS_DIFF_opp,TEAM_COVERED_opp,WL_PCT,ATS_PCT,AVG_SPREAD,AVG_ML,AVG_ATS_DIFF,OREB_PCT_team,OREB_PCT_opp,DREB_PCT_team,DREB_PCT_opp,REB_PCT_team,REB_PCT_opp,TS_PCT_team,TS_PCT_opp,EFG_PCT_team,EFG_PCT_opp,AST_RATIO_team,AST_RATIO_opp,TOV_PCT_team,TOV_PCT_opp,REST,team_elo_pred,team_rating_i,team_rating_n,opp_rating_i,opp_rating_n,elo_MOV_pred
11,2021-22,1610612761,TOR,Toronto Raptors,0022100006,2021-10-20,TOR vs. WAS,1,83,-15,0,1.699301,-4.5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.565538,1454.980195,1439.172768,1478.176334,1493.983760,-0.83
38,2021-22,1610612761,TOR,Toronto Raptors,0022100020,2021-10-22,TOR @ BOS,0,115,32,1,3.200000,6.5,1,0.000000,23.000000,63.000000,7.000000,34.000000,16.000000,22.000000,19.000000,35.000000,54.000000,14.000000,10.000000,4.000000,19.000000,21.000000,83.000000,-15.000000,77.800000,91.200000,-13.400000,107.000000,0.300000,5.000000,18.000000,17.000000,40.000000,8.000000,5.000000,14.000000,1.000000,-19.500000,1.000000,31.000000,54.000000,5.000000,27.000000,...,44.000000,51.000000,20.000000,14.000000,8.000000,22.000000,17.000000,98.000000,15.000000,91.200000,77.800000,13.400000,106.000000,0.700000,5.000000,8.000000,18.000000,55.000000,15.000000,5.000000,15.000000,0.000000,19.500000,1.000000,0.000000,0.000000,-4.500000,1.699301,-19.500000,0.301587,0.166667,0.833333,0.698413,0.514286,0.485714,0.407502,0.565037,0.345361,0.475309,13.084112,18.867925,15.117759,19.224048,NaN,0.331554,1439.172768,1473.120855,1491.977621,1458.029534,-1.89
57,2021-22,1610612761,TOR,Toronto Raptors,0022100029,2021-10-23,TOR vs. DAL,1,95,-8,0,2.550000,4.5,0,0.526316,27.736842,64.052632,8.578947,34.526316,18.631579,21.473684,20.052632,37.105263,57.157895,18.210526,13.157895,2.947368,15.315789,16.789474,99.842105,9.736842,97.221053,86.094737,11.126316,103.842105,0.506842,5.000000,19.578947,22.263158,48.947368,11.157895,6.578947,16.105263,1.526316,11.026316,0.473684,25.736842,50.842105,8.157895,30.684211,...,37.684211,46.263158,19.473684,9.789474,10.105263,23.578947,18.052632,90.105263,-9.736842,86.094737,97.221053,-11.126316,103.368421,0.493158,7.105263,8.000000,12.736842,42.894737,11.842105,7.105263,12.894737,0.526316,-11.026316,0.473684,0.500000,0.500000,1.000000,2.449650,9.500000,0.347311,0.187788,0.812212,0.652689,0.552672,0.447328,0.483248,0.526873,0.411906,0.465784,17.536746,18.839104,12.417219,20.853116,NaN,0.510697,1473.120855,1463.791821,1534.686759,1544.015793,-2.20
91,2021-22,1610612761,TOR,Toronto Raptors,0022100046,2021-10-25,TOR vs. CHI,1,108,-3,0,2.250000,4.0,1,0.332103,25.988930,58.129151,9.472325,35.439114,17.660517,22.036900,16.712177,37.435424,54.147601,17.763838,10.885609,4.073801,14.092251,19.081181,98.055351,3.191882,97.803690,94.362362,3.441328,99.841328,0.484022,6.476015,16.413284,15.154982,44.169742,9.992620,7.103321,15.328413,1.701107,5.666052,0.667897,25.095941,50.162362,

In [40]:
full_df.loc[full_df['TEAM_ABBREVIATION_team'] == 'PHI']

,SEASON_team,TEAM_ID_team,TEAM_ABBREVIATION_team,TEAM_NAME_team,GAME_ID,GAME_DATE_team,MATCHUP_team,HOME_GAME_team,TEAM_SCORE_team,POINT_DIFF_team,WL_team,ML_team,SPREAD_team,TEAM_COVERED_team,RECORD_team,FG2M_team,FG2A_team,FG3M_team,FG3A_team,FTM_team,FTA_team,OREB_team,DREB_team,REB_team,AST_team,STL_team,BLK_team,TOV_team,PF_team,PTS_team,PLUS_MINUS_team,E_OFF_RATING_team,E_DEF_RATING_team,E_NET_RATING_team,POSS_team,PIE_team,PTS_2PT_MR_team,PTS_FB_team,PTS_OFF_TOV_team,PTS_PAINT_team,AST_2PM_team,AST_3PM_team,UAST_2PM_team,UAST_3PM_team,ATS_DIFF_team,RECORD_opp,FG2M_opp,FG2A_opp,FG3M_opp,FG3A_opp,...,DREB_opp,REB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp,PLUS_MINUS_opp,E_OFF_RATING_opp,E_DEF_RATING_opp,E_NET_RATING_opp,POSS_opp,PIE_opp,PTS_2PT_MR_opp,PTS_FB_opp,PTS_OFF_TOV_opp,PTS_PAINT_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp,ATS_DIFF_opp,TEAM_COVERED_opp,WL_PCT,ATS_PCT,AVG_SPREAD,AVG_ML,AVG_ATS_DIFF,OREB_PCT_team,OREB_PCT_opp,DREB_PCT_team,DREB_PCT_opp,REB_PCT_team,REB_PCT_opp,TS_PCT_team,TS_PCT_opp,EFG_PCT_team,EFG_PCT_opp,AST_RATIO_team,AST_RATIO_opp,TOV_PCT_team,TOV_PCT_opp,REST,team_elo_pred,team_rating_i,team_rating_n,opp_rating_i,opp_rating_n,elo_MOV_pred
16,2021-22,1610612755,PHI,Philadelphia 76ers,0022100009,2021-10-20,PHI @ NOP,0,117,20,1,1.555556,-4.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.616381,1622.261094,1634.051624,1470.881545,1459.091015,5.41
41,2021-22,1610612755,PHI,Philadelphia 76ers,0022100021,2021-10-22,PHI vs. BKN,1,109,-5,0,2.300000,1.5,0,1.000000,31.000000,60.000000,13.000000,25.000000,16.000000,20.000000,6.000000,41.000000,47.000000,24.000000,9.000000,5.000000,13.000000,11.000000,117.000000,20.000000,116.100000,96.100000,20.000000,99.000000,0.600000,12.000000,29.000000,18.000000,49.000000,14.000000,9.000000,16.000000,3.000000,15.500000,0.000000,26.000000,63.000000,13.000000,30.000000,...,34.000000,44.000000,28.000000,5.000000,6.000000,14.000000,16.000000,97.000000,-20.000000,96.100000,116.100000,-20.000000,99.000000,0.400000,16.000000,25.000000,9.000000,35.000000,17.000000,10.000000,8.000000,2.000000,-15.500000,0.000000,1.000000,1.000000,-4.500000,1.555556,15.500000,0.150000,0.196078,0.803922,0.850000,0.516484,0.483516,0.654362,0.510634,0.594118,0.489247,24.242424,28.282828,12.172285,12.617159,NaN,0.633300,1634.051624,1624.405619,1608.132557,1617.778562,0.93
74,2021-22,1610612755,PHI,Philadelphia 76ers,0022100038,2021-10-24,PHI @ OKC,0,115,12,1,1.277778,-8.5,1,0.473684,29.947368,57.368421,13.526316,30.789474,12.315789,15.263158,5.473684,35.210526,40.684211,20.315789,8.473684,7.105263,11.421053,16.263158,112.789474,6.842105,111.889474,105.205263,6.684211,97.947368,0.520526,14.631579,22.684211,18.000000,44.263158,11.894737,7.947368,17.052632,5.105263,5.500000,0.526316,27.578947,56.157895,12.473684,34.210526,...,35.052632,45.578947,26.947368,5.526316,6.000000,13.473684,16.000000,105.947368,-6.842105,105.205263,111.889474,-6.684211,97.947368,0.479474,17.052632,17.631579,7.947368,37.631579,17.526316,7.894737,9.052632,3.578947,-5.500000,0.526316,0.500000,0.500000,-1.500000,1.927778,6.000000,0.135065,0.230150,0.769850,0.864935,0.471629,0.528371,0.616230,0.563039,0.569851,0.512231,20.741537,27.512090,10.744702,12.108370,NaN,0.840890,1624.405619,1627.412733,1266.189114,1263.182000,12.79
98,2021-22,1610612755,PHI,Philadelphia 76ers,0022100050,2021-10-26,PHI @ NYK,0,99,-13,0,1.952381,3.0,0,0.667897,27.752768,53.542435,14.808118,34.557196,13.675277,17.011070,7.881919,35.132841,43.014760,22.044280,6.822878,7.804428,11.634686,16.535055,113.605166,8.745387,114.439483,105.424723,9.014760,97.597786,0.520332,12.922509,14.313653,15.047970,41.583026,11.195572,10.180812,15.557196,3.959410,4.761993,0.332103,29.2

In [42]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.16it/s]
progress:: 340it [00:00, 5000.20it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-11-12,CHA,NYK,1.5,0.517374,0.481871,0.517751
1,2021-11-12,BOS,MIL,1.5,0.503387,0.497226,0.503081
2,2021-11-12,CLE,DET,-4.5,0.404828,0.594824,0.405002
3,2021-11-12,HOU,POR,6.5,0.452335,0.549734,0.451300
4,2021-11-12,OKC,SAC,4.5,0.503852,0.495762,0.504045
5,2021-11-12,MEM,PHX,2.0,0.472050,0.526361,0.472845
6,2021-11-12,NOP,BKN,9.0,0.567230,0.433053,0.567088
7,2021-11-12,SAS,DAL,3.0,0.503857,0.496352,0.503753
8,2021-11-12,DEN,ATL,-4.0,0.527909,0.473227,0.527341
9,2021-11-12,GSW,CHI,-5.5,0.486819,0.513113,0.486853


In [43]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_sfs')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]
progress:: 340it [00:00, 5000.17it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-11-12,CHA,NYK,1.5,0.524984,0.483803,0.520591
1,2021-11-12,BOS,MIL,1.5,0.517213,0.494285,0.511464
2,2021-11-12,CLE,DET,-4.5,0.412579,0.599824,0.406378
3,2021-11-12,HOU,POR,6.5,0.462633,0.549772,0.456430
4,2021-11-12,OKC,SAC,4.5,0.500860,0.511653,0.494604
5,2021-11-12,MEM,PHX,2.0,0.473531,0.535217,0.469157
6,2021-11-12,NOP,BKN,9.0,0.539974,0.465913,0.537030
7,2021-11-12,SAS,DAL,3.0,0.510449,0.498772,0.505839
8,2021-11-12,DEN,ATL,-4.0,0.521797,0.493931,0.513933
9,2021-11-12,GSW,CHI,-5.5,0.523503,0.494411,0.514546


In [ ]:
connection = sqlite3.connect('../data/nba.db')

pd.read_sql("SELECT * FROM ")